In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import shutil
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
from PIL import Image
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
base_dir = '/content/drive/MyDrive/Image Classification/Images'
train_dir = '/content/drive/MyDrive/Image Classification/Images/Train'
train_saree = '/content/drive/MyDrive/Image Classification/Images/Train/Saree Train'
train_jeans = '/content/drive/MyDrive/Image Classification/Images/Train/Jeans Train'
train_trousers = '/content/drive/MyDrive/Image Classification/Images/Train/Trouser Train'
test_dir = '/content/drive/MyDrive/Image Classification/Images/Test'
test_saree_dir = '/content/drive/MyDrive/Image Classification/Images/Test/Saree Test'
test_jeans_dir = '/content/drive/MyDrive/Image Classification/Images/Test/Jeans Test'
test_trousers_dir = '/content/drive/MyDrive/Image Classification/Images/Test/Trouser Test'
Validation_dir = '/content/drive/MyDrive/Image Classification/Images/Validation'
valid_saree_dir = '/content/drive/MyDrive/Image Classification/Images/Validation/Saree Validation'
valid_jeans_dir = '/content/drive/MyDrive/Image Classification/Images/Validation/Jeans Validation'
valid_trousers_dir = '/content/drive/MyDrive/Image Classification/Images/Validation/Trouser Validation'

In [ ]:
saree_train = len(os.listdir(train_saree))
jeans_train = len(os.listdir(train_jeans))
trousers_train = len(os.listdir(train_trousers))
saree_test = len(os.listdir(test_saree_dir))
jeans_test = len(os.listdir(test_jeans_dir))
trousers_test = len(os.listdir(test_trousers_dir))
saree_valid = len(os.listdir(valid_saree_dir))
jeans_valid = len(os.listdir(valid_jeans_dir))
trousers_valid = len(os.listdir(valid_trousers_dir))

In [ ]:
total_train = saree_train+jeans_train+trousers_train
total_validation = saree_valid+jeans_valid+trousers_valid
total_test = saree_test+jeans_test+trousers_test

In [ ]:
IMG_SHAPE  = 224
batch_size = 32

In [ ]:
image_gen_train = ImageDataGenerator(rescale = 1./255)
train_data_gen = image_gen_train.flow_from_directory(batch_size = batch_size,
directory = train_dir,
shuffle= True,
target_size = (IMG_SHAPE,IMG_SHAPE),
class_mode = 'categorical')
image_generator_validation = ImageDataGenerator(rescale=1./255)
val_data_gen = image_generator_validation.flow_from_directory(batch_size=batch_size,
directory=test_dir,
target_size=(IMG_SHAPE, IMG_SHAPE),
class_mode='categorical')
image_gen_test = ImageDataGenerator(rescale=1./255)
test_data_gen = image_gen_test.flow_from_directory(batch_size=batch_size,
directory=Validation_dir,
target_size=(IMG_SHAPE, IMG_SHAPE),
class_mode='categorical')

Found 491 images belonging to 3 classes.
Found 115 images belonging to 3 classes.
Found 114 images belonging to 3 classes.


In [ ]:
pre_trained_model = tf.keras.applications.VGG16(input_shape=(224,224, 3), include_top=False, weights="imagenet")

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
for layer in pre_trained_model.layers:
  print(layer.name)
layer.trainable = False

input_1
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool


In [ ]:
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output
x = tf.keras.layers.GlobalMaxPooling2D()(last_output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(3, activation='softmax')(x)

In [ ]:
model = tf.keras.Model(pre_trained_model.input, x)

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.categorical_crossentropy, metrics=['acc'])


In [ ]:
vgg_classifier = model.fit(train_data_gen,
steps_per_epoch=(total_train//batch_size),
epochs = 5,
validation_data=val_data_gen,
validation_steps=(total_validation//batch_size),
batch_size = batch_size,
verbose = 2)

Epoch 1/5
15/15 - 892s - loss: 1.1471 - acc: 0.4031 - val_loss: 1.0406 - val_acc: 0.2812 - 892s/epoch - 59s/step
Epoch 2/5
15/15 - 882s - loss: 0.9825 - acc: 0.4619 - val_loss: 0.8958 - val_acc: 0.5625 - 882s/epoch - 59s/step
Epoch 3/5
15/15 - 882s - loss: 0.7377 - acc: 0.6231 - val_loss: 0.9828 - val_acc: 0.4896 - 882s/epoch - 59s/step
Epoch 4/5
15/15 - 903s - loss: 0.7401 - acc: 0.5991 - val_loss: 0.7613 - val_acc: 0.5729 - 903s/epoch - 60s/step
Epoch 5/5
15/15 - 882s - loss: 0.6512 - acc: 0.6558 - val_loss: 0.6114 - val_acc: 0.6458 - 882s/epoch - 59s/step


In [ ]:
result = model.evaluate(test_data_gen,batch_size=batch_size)
print("test_loss, test accuracy",result)

4/4 [==============================] - 56s 13s/step - loss: 0.5917 - acc: 0.7281
test_loss, test accuracy [0.5916635394096375, 0.7280701994895935]
